In [5]:
import os
import h5py
import argparse
import numpy as np

from hdf5_utils import *

cars_hdf5_name = '../models/cars/2_watertight/sdf.h5'

h5 = read_hdf5(cars_hdf5_name)
print h5.shape

S = h5.reshape((h5.shape[0], -1))
print S.shape

S_train = S[:74, :]
S_test = S[-5:, :]

(79, 32, 32, 32)
(79, 32768)


In [6]:
S_mean = np.mean(S_train, axis=0)
S_white_train = S_train - S_mean
S_white_test = S_test - S_mean
print S_white_train.shape, S_white_test.shape, S_mean.shape
# t = S.shape[0] # number of samples, 87
# n = S.shape[1] # dim of feature, 32*32*32

from sklearn import decomposition
pca = decomposition.PCA(n_components=10)
pca.fit(S_white_train)
V = pca.components_.T # basis (mapping from high to low dim feature space), 32786*10

print V.shape

(74, 32768) (5, 32768) (32768,)
(32768, 10)


In [8]:
beta = np.dot(V.T, S_white_test.T) # embedding for 87 models, 10*87
print beta.shape

S_recover = np.dot(V, beta).T+S_mean.reshape((1, -1))
print S_recover.shape

hdf5_recovered_path = '../models/cars/2_watertight/recovered/recovered_test_recon.h5'
write_hdf5(hdf5_recovered_path, S_recover.reshape((-1, 32, 32, 32)))
mc_recovered_path = hdf5_recovered_path.replace('recovered_test_recon.h5', 'mc_test_recon')
os.system('python marching_cubes.py %s %s'%(hdf5_recovered_path, mc_recovered_path))

hdf5_recovered_path = '../models/cars/2_watertight/recovered/recovered_test.h5'
write_hdf5(hdf5_recovered_path, S_test.reshape((-1, 32, 32, 32)))
mc_recovered_path = hdf5_recovered_path.replace('recovered_test.h5', 'mc_test')
os.system('python marching_cubes.py %s %s'%(hdf5_recovered_path, mc_recovered_path))

(10, 5)
(5, 32768)


0